In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1 # dropout大于等于0，小于等于1，否则报错
    if dropout == 1:
        return torch.zeros_like(X) # 如果dropout为1，则X返回为全0
    if dropout == 0:
        return X # 如果dropout为1，则X返回为全原值
    mask = (torch.randn(X.shape)>dropout).float() # 取X.shape里面0到1之间的均匀分布，如果值大于dropout，则把它选出来
    #print((torch.randn(X.shape)>dropout)) # 返回的是布尔值，然后转布尔值为0、1
    return mask * X / (1.0 - dropout) 

In [3]:
x = torch.arange(16,dtype=torch.float32).reshape((2,8))
print(dropout_layer(x,0.4))

tensor([[ 0.0000,  0.0000,  0.0000,  5.0000,  6.6667,  8.3333,  0.0000, 11.6667],
        [13.3333, 15.0000,  0.0000,  0.0000,  0.0000, 21.6667,  0.0000, 25.0000]])


In [5]:
# 定义具有两个隐藏层的多层感知机，每个隐藏层包含256个单元
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10 ,256, 256

dropout1, dropout2 = 0.2, 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens1, num_hiddens2,is_training=True):       
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU()
        
    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1,self.num_inputs))))
        if self.training == True: # 如果是在训练，则作用dropout，否则则不作用
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            H2 = dropout_layer(H2,dropout2)
        out = self.lin3(H2) # 输出层不作用dropout
        return out
    
net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)

In [7]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer):
    # 创建动画器用于可视化训练进度
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    
    # 存储训练过程中每个epoch的指标
    metrics = {'train_loss': [], 'train_acc': [], 'test_acc': []}
    
    # 训练循环
    for epoch in range(num_epochs):
        # 训练指标
        train_metrics = train_epoch_ch3(net, train_iter, loss, trainer)
        
        # 计算测试准确率
        test_acc = evaluate_accuracy(net, test_iter)
        
        # 存储指标
        metrics['train_loss'].append(train_metrics[0])
        metrics['train_acc'].append(train_metrics[1])
        metrics['test_acc'].append(test_acc)
        
        # 更新动画
        animator.add(epoch + 1, 
                    [metrics['train_loss'][-1], metrics['train_acc'][-1], 
                     metrics['test_acc'][-1]])
        
        # 打印进度
        print(f'epoch {epoch+1}, '
              f'loss {train_metrics[0]:.3f}, '
              f'train acc {train_metrics[1]:.3f}, '
              f'test acc {test_acc:.3f}')
    
    # 返回最终指标
    return metrics

def train_epoch_ch3(net, train_iter, loss, trainer):
    """训练一个epoch"""
    # 训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    
    # 指标累加器
    metric_sum = torch.tensor([0.0, 0.0])  # 损失总和, 准确预测的样本数
    sample_count = 0  # 样本总数
    
    for X, y in train_iter:
        # 前向传播
        y_hat = net(X)
        l = loss(y_hat, y)
        
        # 梯度清零
        if isinstance(trainer, torch.optim.Optimizer):
            trainer.zero_grad()
        elif isinstance(net, torch.nn.Module):
            net.zero_grad()
        else:
            pass  # 如果是自定义训练器，可能需要其他处理
            
        # 反向传播
        l.mean().backward()
        
        # 参数更新
        trainer.step()
        
        # 更新指标
        metric_sum[0] += float(l.sum())
        metric_sum[1] += float(accuracy(y_hat, y))
        sample_count += y.numel()
    
    # 返回训练损失和训练准确率
    return metric_sum[0] / sample_count, metric_sum[1] / sample_count

def evaluate_accuracy(net, data_iter):
    """评估模型在数据集上的准确率"""
    # 评估模式
    if isinstance(net, torch.nn.Module):
        net.eval()  
    
    # 指标累加器
    metric = Accumulator(2)  # 正确预测的数量, 总预测的数量
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

def accuracy(y_hat, y):
    """计算预测正确的数量"""
    # 对于分类问题，使用argmax获取预测类别
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    # 将预测类别与真实类别比较
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

class Animator:
    """用于动态可视化训练进度的动画器"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 创建图表和轴
        if legend is None:
            legend = []
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes,]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: self._config_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
        self.config_axes()
        
    def _config_axes(self, axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
        """配置坐标轴"""
        axes.set_xlabel(xlabel)
        axes.set_ylabel(ylabel)
        axes.set_xscale(xscale)
        axes.set_yscale(yscale)
        axes.set_xlim(xlim)
        axes.set_ylim(ylim)
        if legend:
            axes.legend(legend)
        axes.grid()
        
    def add(self, x, y):
        """添加数据点"""
        if not hasattr(y, '__len__'):
            y = [y]
        n = len(y)
        if not hasattr(x, '__len__'):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        plt.draw()
        plt.pause(0.001)
        display.display(self.fig)
        display.clear_output(wait=True)

class Accumulator:
    """用于累积训练指标的累加器"""
    def __init__(self, n):
        self.data = [0.0] * n
        
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
        
    def reset(self):
        self.data = [0.0] * len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx]

In [6]:
# 训练和测试
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(),lr=lr)
train_ch3(net,train_iter,test_iter,loss,num_epochs,trainer)

AttributeError: module 'd2l.torch' has no attribute 'train_ch3'